In [2]:
import pandas as pd
import streamlit as st 
import plotly.express as px

from funcoes import *

In [4]:
dfT2024 = process_file_for_tec("dados/2024-1_GestaoResultado_ResumoInscricoes_TEC.xlsx")
# dfS2024 = process_file_for_sub("dados/2024-1_GestaoResultado_ResumoInscricoes_SUB.xlsx")
dfG2024 = process_file_for_grad("dados/2024-1_GestaoResultado_ResumoInscricoes_GRAD.xlsx")

dfT2025 = process_file_for_tec("dados/2025-1_GestaoResultado_ResumoInscricoes_TEC.xlsx")
dfS2025 = process_file_for_sub("dados/2025-1_GestaoResultado_ResumoInscricoes_SUB.xlsx")
dfG2025 = process_file_for_grad("dados/2025-1_GestaoResultado_ResumoInscricoes_GRAD.xlsx")



In [5]:
dfAll2024 = pd.concat([dfT2024, dfG2024])
dfAll2025 = pd.concat([dfT2025, dfS2025, dfG2025])


In [4]:
dfAll2024

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas,FormaIngresso
0,ADMINISTRAÇÃO,Integrado,FORMIGA,Turno Integral,Nível Fundamental,84,54,14,68,Processo Seletivo
1,ADMINISTRAÇÃO,Integrado,BAMBUÍ,Turno Integral,Nível Fundamental,66,47,4,51,Processo Seletivo
2,ADMINISTRAÇÃO,Integrado,OURO BRANCO,Turno Integral,Nível Fundamental,221,171,25,196,Processo Seletivo
3,ADMINISTRAÇÃO,Integrado,OURO PRETO,Turno Integral,Nível Fundamental,273,223,22,245,Processo Seletivo
4,ADMINISTRAÇÃO,Integrado,PONTE NOVA,Turno Integral,Nível Fundamental,339,282,28,310,Processo Seletivo
...,...,...,...,...,...,...,...,...,...,...
66,SISTEMAS DE INFORMAÇÃO,Bacharelado,OURO BRANCO,Turno Noturno,Nível Médio,108,64,22,86,Aproveitamento de notas obtidas no ENEM
67,SISTEMAS DE INFORMAÇÃO,Bacharelado,SABARÁ,Turno Matutino,Nível Médio,120,73,21,94,Aproveitamento de notas obtidas no ENEM
68,PROCESSOS GERENCIAIS,Tecnológico,PONTE NOVA,Turno Noturno,Nível Médio,23,8,4,12,Analise de Desempenho Escolar mais uma prova d...
69,PROCESSOS GERENCIAIS,Tecnológico,PONTE NOVA,Turno Noturno,Nível Médio,35,11,8,19,Aproveitamento de notas obtidas no ENEM


In [5]:
dfAll2025

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas,FormaIngresso
0,ADMINISTRAÇÃO,Integrado,ARCOS,Turnos Manhã e Tarde,Médio,48,25,0,25,Processo Seletivo
1,MECÂNICA,Integrado,ARCOS,Turnos Manhã e Tarde,médio,17,11,0,11,Processo Seletivo
2,ADMINISTRAÇÃO,Integrado,BAMBUÍ,Turnos Manhã e Tarde,médio,11,6,0,6,Processo Seletivo
3,AGROPECUÁRIA,Integrado,BAMBUÍ,Turnos Manhã e Tarde,médio,40,18,0,18,Processo Seletivo
4,BIOTECNOLOGIA,Integrado,BAMBUÍ,Turnos Manhã e Tarde,médio,15,8,0,8,Processo Seletivo
...,...,...,...,...,...,...,...,...,...,...
83,MATEMÁTICA,Licenciatura,SÃO JOÃO EVANGELISTA,Turno Noturno,Superior,1,0,0,0,Notas do Enem
84,MATEMÁTICA,Licenciatura,SÃO JOÃO EVANGELISTA,Turno Noturno,Superior,0,0,0,0,Redação Online
85,SISTEMAS DE INFORMAÇÃO,Bacharelado,SÃO JOÃO EVANGELISTA,Turno Manhã e Tarde,Superior,3,1,0,1,Notas do Enem
86,SISTEMAS DE INFORMAÇÃO,Bacharelado,SÃO JOÃO EVANGELISTA,Turno Manhã e Tarde,Superior,0,0,0,0,Redação Online


In [26]:
dfx = dfAll2025.groupby(["Modalidade","Curso"])[["Inscritos"]].sum().reset_index()
dfx[(dfx['Modalidade'] != "Integrado") & (dfx['Modalidade'] != "Subsequente")].sort_values(by='Modalidade')

,Modalidade,Curso,Inscritos
0,Bacharelado,ADMINISTRAÇÃO,88
16,Bacharelado,ZOOTECNIA,5
14,Bacharelado,MEDICINA VETERINÁRIA,66
13,Bacharelado,ENGENHARIA METALÚRGICA,20
12,Bacharelado,ENGENHARIA MECÂNICA,48
11,Bacharelado,ENGENHARIA FLORESTAL,1
10,Bacharelado,ENGENHARIA ELÉTRICA,37
9,Bacharelado,ENGENHARIA DE PRODUÇÃO,22
15,Bacharelado,SISTEMAS DE INFORMAÇÃO,51
7,Bacharelado,ENGENHARIA DE ALIMENTOS,0


In [29]:
lista_licenciatura = ["PEDAGOGIA","MATEMÁTICA","FÍSICA","EDUCAÇÃO FÍSICA","CIÊNCIAS BIOLÓGICAS","GEOGRAFIA"]
lista_tecnologico =  ["LOGÍSTICA","ANÁLISE E DESENVOLVIMENTO DE SISTEMAS","CONSERVAÇÃO E RESTAURO","DESIGN DE INTERIORES","GESTÃO AMBIENTAL","GESTÃO DA QUALIDADE","PROCESSOS GERENCIAIS"]

def tipo_curso(x):
    if x in lista_licenciatura: return "Licenciatura"
    if x in lista_tecnologico: return "Tecnológico"
    return "Bacharelado"

In [30]:
dfAll2024["Modalidade"] =  dfAll2024["Curso"].apply(tipo_curso)

0     Bacharelado
1     Bacharelado
2     Bacharelado
3     Bacharelado
4     Bacharelado
         ...     
66    Bacharelado
67    Bacharelado
68    Tecnológico
69    Tecnológico
70    Bacharelado
Name: Curso, Length: 142, dtype: object

In [6]:
dfAll2024.groupby("Modalidade")[["Inscritos"]].sum().reset_index().sort_values(by='Inscritos', ascending=False)

,Modalidade,Inscritos
2,Integrado,13574
1,Bacharelado,2985
4,Subsequente,1319
5,Tecnológico,390
3,Licenciatura,300
0,-1,3


In [7]:
dfAll2025.groupby("Modalidade")[["Inscritos"]].sum().reset_index().sort_values(by='Inscritos', ascending=False)

,Modalidade,Inscritos
1,Integrado,2393
0,Bacharelado,554
3,Subsequente,276
4,Tecnológico,74
2,Licenciatura,39


In [10]:
diff(dfAll2024,dfAll2025, "Modalidade")

,Modalidade,Inscritos,Pagos,Deferidas,Homologadas
2,Integrado,-11181.0,-9452.0,-1392.0,-10844.0
1,Bacharelado,-2431.0,-1340.0,-549.0,-1889.0
4,Subsequente,-1043.0,-383.0,-152.0,-535.0
5,Tecnológico,-316.0,-131.0,-78.0,-209.0
3,Licenciatura,-261.0,-133.0,-69.0,-202.0
0,-1,NaN,NaN,NaN,NaN


In [7]:
dfAll2025

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas,FormaIngresso
0,ADMINISTRAÇÃO,Integrado,ARCOS,Turnos Manhã e Tarde,Médio,51,29,0,29,Processo Seletivo
1,MECÂNICA,Integrado,ARCOS,Turnos Manhã e Tarde,médio,17,14,0,14,Processo Seletivo
2,ADMINISTRAÇÃO,Integrado,BAMBUÍ,Turnos Manhã e Tarde,médio,12,6,0,6,Processo Seletivo
3,AGROPECUÁRIA,Integrado,BAMBUÍ,Turnos Manhã e Tarde,médio,47,22,0,22,Processo Seletivo
4,BIOTECNOLOGIA,Integrado,BAMBUÍ,Turnos Manhã e Tarde,médio,15,8,0,8,Processo Seletivo
...,...,...,...,...,...,...,...,...,...,...
83,MATEMÁTICA,Licenciatura,SÃO JOÃO EVANGELISTA,Turno Noturno,Superior,1,0,0,0,Notas do Enem
84,MATEMÁTICA,Licenciatura,SÃO JOÃO EVANGELISTA,Turno Noturno,Superior,0,0,0,0,Redação Online
85,SISTEMAS DE INFORMAÇÃO,Bacharelado,SÃO JOÃO EVANGELISTA,Turno Manhã e Tarde,Superior,3,1,0,1,Notas do Enem
86,SISTEMAS DE INFORMAÇÃO,Bacharelado,SÃO JOÃO EVANGELISTA,Turno Manhã e Tarde,Superior,0,0,0,0,Redação Online


In [29]:
soma_colunas = dfAll2025[["Inscritos", "Pagos", "Deferidas", "Homologadas"]].sum()
df_soma = pd.DataFrame(soma_colunas, columns=["total"]).reset_index()
df_soma

,index,total
0,Inscritos,3655
1,Pagos,1365
2,Deferidas,0
3,Homologadas,1365


In [30]:
fig = px.pie(df_soma, values='total', names='index', title='Population of European continent')
fig.show()


In [11]:
df = pd.read_excel("dados/processed/all_data.xlsx")
df.head(10)
print(df.shape)

(1248, 11)


In [12]:
df.fillna(0, inplace=True)

In [13]:
campus = "PONTE NOVA"
df_filtered = df[(df["Modalidade"] == "Integrado") & (df["Campus"] == campus)]
df_filtered

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,FormaIngresso,Isenções deferidas,Inscrições homologadas,Timestamp
37,ADMINISTRAÇÃO,Integrado,PONTE NOVA,Turnos Manhã e Tarde,médio,73,29,Processo Seletivo,0,29,2024-09-03 15:49:27.870
38,INFORMÁTICA,Integrado,PONTE NOVA,Turnos Manhã e Tarde,médio,94,33,Processo Seletivo,0,33,2024-09-03 15:49:27.870
298,ADMINISTRAÇÃO,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,74,29,Processo Seletivo,0,29,2024-09-03 16:13:53.915
299,INFORMÁTICA,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,95,33,Processo Seletivo,0,33,2024-09-03 16:13:53.915
437,ADMINISTRAÇÃO,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,77,29,Processo Seletivo,0,29,2024-09-03 19:45:51.682
438,INFORMÁTICA,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,99,34,Processo Seletivo,0,34,2024-09-03 19:45:51.682
505,ADMINISTRAÇÃO,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,78,29,Processo Seletivo,0,29,2024-09-03 21:03:30.731
506,INFORMÁTICA,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,101,34,Processo Seletivo,0,34,2024-09-03 21:03:30.731
661,ADMINISTRAÇÃO,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,78,29,Processo Seletivo,0,29,2024-09-03 23:05:14.964
662,INFORMÁTICA,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,102,34,Processo Seletivo,0,34,2024-09-03 23:05:14.964


In [16]:
x = ["Inscritos","Pagos", "Isenções deferidas","Inscrições homologadas"],
situacao = x[0]

fig22 = px.line(df_filtered, title=f"Campus {campus}", x="Timestamp", y="Inscrições homologadas", color="Curso", text="Inscrições homologadas")
fig22.update_xaxes(title='')
fig22.update_yaxes(tickformat=",d")
# Exibe os valores no gráfico
fig22.update_traces(texttemplate='%{text:.0f}', textposition="top right")
fig22.update_layout(hovermode="x") 

In [69]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup

In [70]:
page = requests.get("https://pt.wikipedia.org/wiki/Lista_de_municípios_do_Brasil_por_população_(2022)")

In [71]:
soup = soup(page.content, 'html.parser')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
# transformar a coluna população em int
df["População"] = df["População"].str.replace("\xa0","").astype(int)
df["Município"] = df["Município"].str.replace("[nota 1]","")

In [73]:
df_proc = pd.read_excel("D:/Dev/DataScience/vestibular3/dados/processado.xlsx")
df_proc 

,Inscricao,Cidade,Campus,Curso,Cidade_UF,Campus_UF
0,438121,Terra Nova,Betim,Técnico Integrado em Automação Industrial,Terra Nova - BA,Betim - MG
1,438123,Ponte Nova,Ponte Nova,Técnico Integrado em Administração,Ponte Nova - MG,Ponte Nova - MG
2,438125,Governador Valadares,Governador Valadares,Técnico Integrado em Edificações,Governador Valadares - MG,Governador Valadares - MG
3,438127,Contagem,Betim,Técnico Integrado em Mecânica,Contagem - MG,Betim - MG
4,438128,Sabará,Sabará,Técnico Integrado em Administração,Sabará - MG,Sabará - MG
...,...,...,...,...,...,...
8741,446861,Congonhas,Congonhas,Técnico Subsequente em Mineração,Congonhas - MG,Congonhas - MG
8742,446866,Governador Valadares,Ribeirão das Neves,Técnico Subsequente em Logística,Governador Valadares - MG,Ribeirão das Neves - MG
8743,446895,Belo Horizonte,Santa Luzia,Técnico Subsequente em Segurança do Trabalho,Belo Horizonte - MG,Santa Luzia - MG
8744,446914,Caraguatatuba,Ribeirão das Neves,Técnico Subsequente em Logística,Caraguatatuba - SP,Ribeirão das Neves - MG


In [86]:
df_total = df_proc["Campus"].value_counts().to_frame().reset_index()
df_total

,Campus,count
0,Betim,1042
1,Ribeirão das Neves,883
2,Ouro Preto,860
3,Sabará,700
4,Santa Luzia,674
5,São João Evangelista,663
6,Ouro Branco,536
7,Ibirité,491
8,Governador Valadares,455
9,Bambuí,453


In [95]:
df_filter = df[ (df["Município"].isin(df_total["Campus"])) & (df["Unidade federativa"] == "Minas Gerais") ].copy()
df_filter

,Posição,Código IBGE,Município,Unidade federativa,População
59,60º,3106705,Betim,Minas Gerais,411846
83,84º,3154606,Ribeirão das Neves,Minas Gerais,329794
110,111º,3127701,Governador Valadares,Minas Gerais,257171
132,133º,3131307,Ipatinga,Minas Gerais,227731
142,143º,3157807,Santa Luzia,Minas Gerais,219132
170,171º,3129806,Ibirité,Minas Gerais,170537
226,227º,3118304,Conselheiro Lafaiete,Minas Gerais,131621
233,234º,3156700,Sabará,Minas Gerais,129380
433,434º,3146107,Ouro Preto,Minas Gerais,74821
481,482º,3126109,Formiga,Minas Gerais,68248


In [97]:
df_merged = df_filter.merge(df_total, left_on="Município", right_on="Campus", how="inner")
df_merged

,Posição,Código IBGE,Município,Unidade federativa,População,Campus,count
0,60º,3106705,Betim,Minas Gerais,411846,Betim,1042
1,84º,3154606,Ribeirão das Neves,Minas Gerais,329794,Ribeirão das Neves,883
2,111º,3127701,Governador Valadares,Minas Gerais,257171,Governador Valadares,455
3,133º,3131307,Ipatinga,Minas Gerais,227731,Ipatinga,398
4,143º,3157807,Santa Luzia,Minas Gerais,219132,Santa Luzia,674
5,171º,3129806,Ibirité,Minas Gerais,170537,Ibirité,491
6,227º,3118304,Conselheiro Lafaiete,Minas Gerais,131621,Conselheiro Lafaiete,260
7,234º,3156700,Sabará,Minas Gerais,129380,Sabará,700
8,434º,3146107,Ouro Preto,Minas Gerais,74821,Ouro Preto,860
9,482º,3126109,Formiga,Minas Gerais,68248,Formiga,206


In [102]:
df_merged["Can_Pop"] = df_merged["count"] / df_merged["População"] * 100


In [103]:
df_merged

,Posição,Código IBGE,Município,Unidade federativa,População,Campus,count,Can_Pop
0,60º,3106705,Betim,Minas Gerais,411846,Betim,1042,0.253007
1,84º,3154606,Ribeirão das Neves,Minas Gerais,329794,Ribeirão das Neves,883,0.267743
2,111º,3127701,Governador Valadares,Minas Gerais,257171,Governador Valadares,455,0.176925
3,133º,3131307,Ipatinga,Minas Gerais,227731,Ipatinga,398,0.174768
4,143º,3157807,Santa Luzia,Minas Gerais,219132,Santa Luzia,674,0.307577
5,171º,3129806,Ibirité,Minas Gerais,170537,Ibirité,491,0.287914
6,227º,3118304,Conselheiro Lafaiete,Minas Gerais,131621,Conselheiro Lafaiete,260,0.197537
7,234º,3156700,Sabará,Minas Gerais,129380,Sabará,700,0.541042
8,434º,3146107,Ouro Preto,Minas Gerais,74821,Ouro Preto,860,1.149410
9,482º,3126109,Formiga,Minas Gerais,68248,Formiga,206,0.301840


In [104]:
df_merged.style.format({"Can_Pop": "{:.2f}%"})

,Posição,Código IBGE,Município,Unidade federativa,População,Campus,count,Can_Pop
0,60º,3106705,Betim,Minas Gerais,411846,Betim,1042,0.25%
1,84º,3154606,Ribeirão das Neves,Minas Gerais,329794,Ribeirão das Neves,883,0.27%
2,111º,3127701,Governador Valadares,Minas Gerais,257171,Governador Valadares,455,0.18%
3,133º,3131307,Ipatinga,Minas Gerais,227731,Ipatinga,398,0.17%
4,143º,3157807,Santa Luzia,Minas Gerais,219132,Santa Luzia,674,0.31%
5,171º,3129806,Ibirité,Minas Gerais,170537,Ibirité,491,0.29%
6,227º,3118304,Conselheiro Lafaiete,Minas Gerais,131621,Conselheiro Lafaiete,260,0.20%
7,234º,3156700,Sabará,Minas Gerais,129380,Sabará,700,0.54%
8,434º,3146107,Ouro Preto,Minas Gerais,74821,Ouro Preto,860,1.15%
9,482º,3126109,Formiga,Minas Gerais,68248,Formiga,206,0.30%


In [15]:
import pandas as pd

def amostrar_por_campus_com_timestamp(df, n_amostras):
    # Função auxiliar para amostrar mantendo o primeiro e último Timestamp
    def amostrar_grupo(grupo):
        # Ordena o grupo por Timestamp
        grupo = grupo.sort_values(by='Timestamp')
        
        # Garante o primeiro e último valor
        first = grupo.iloc[[0]]
        last = grupo.iloc[[-1]]
        
        # Se o grupo tem mais de 2 linhas, amostra o restante sem incluir o primeiro e o último
        if len(grupo) > 2:
            middle = grupo.iloc[1:-1].sample(min(len(grupo) - 2, n_amostras), random_state=42)
            return pd.concat([first, middle, last])
        else:
            # Se o grupo tem 2 ou menos linhas, apenas retorna o grupo
            return grupo
    
    # Agrupa por 'Campus' e aplica a amostragem para cada grupo
    amostrado = df.groupby('Campus', group_keys=False).apply(amostrar_grupo).reset_index(drop=True)
    
    return amostrado.sort_values(by='Timestamp')

In [10]:
df_all = pd.read_excel("dados/processed/all_data.xlsx")
df_all.shape

(14162, 11)

In [14]:
df_all

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,FormaIngresso,Isenções deferidas,Inscrições homologadas,Timestamp
0,ADMINISTRAÇÃO,Integrado,ARCOS,Turnos Manhã e Tarde,Médio,54,34,Processo Seletivo,0,34,2024-09-03 15:49:27.870
1,MECÂNICA,Integrado,ARCOS,Turnos Manhã e Tarde,médio,20,17,Processo Seletivo,0,17,2024-09-03 15:49:27.870
2,ADMINISTRAÇÃO,Integrado,BAMBUÍ,Turnos Manhã e Tarde,médio,14,7,Processo Seletivo,0,7,2024-09-03 15:49:27.870
3,AGROPECUÁRIA,Integrado,BAMBUÍ,Turnos Manhã e Tarde,médio,53,29,Processo Seletivo,0,29,2024-09-03 15:49:27.870
4,BIOTECNOLOGIA,Integrado,BAMBUÍ,Turnos Manhã e Tarde,médio,16,9,Processo Seletivo,0,9,2024-09-03 15:49:27.870
...,...,...,...,...,...,...,...,...,...,...,...
14157,EM MINERAÇÃO,Subsequente,OURO PRETO,Turno Noturno,Médio,36,17,Processo Seletivo,0,17,2024-09-25 12:00:44.820
14158,EM SEGURANÇA DO TRABALHO,Subsequente,OURO PRETO,Turno Noturno,Médio,89,43,Processo Seletivo,0,43,2024-09-25 12:00:44.820
14159,EM LOGÍSTICA,Subsequente,RIBEIRÃO DAS NEVES,EAD,Médio,301,84,Processo Seletivo,0,84,2024-09-25 12:00:44.820
14160,EM PAISAGISMO,Subsequente,SANTA LUZIA,Turno Noturno,Médio,20,12,Processo Seletivo,0,12,2024-09-25 12:00:44.820


In [20]:
df_amostra = amostrar_por_campus_com_timestamp(df_all, 100)
df_amostra

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,FormaIngresso,Isenções deferidas,Inscrições homologadas,Timestamp
0,ADMINISTRAÇÃO,Integrado,ARCOS,Turnos Manhã e Tarde,Médio,54,34,Processo Seletivo,0,34,2024-09-03 15:49:27.870
1,MECÂNICA,Integrado,ARCOS,Turnos Manhã e Tarde,Médio,22,19,Processo Seletivo,0,19,2024-09-05 08:35:48.605
2,ENGENHARIA MECÂNICA,Bacharelado,ARCOS,Turno Noite,Superior,35,22,Redação Online,0,22,2024-09-12 18:00:47.757
3,ENGENHARIA MECÂNICA,Bacharelado,ARCOS,Turno Noite,Superior,30,16,Redação Online,0,16,2024-09-09 18:02:00.393
4,ENGENHARIA MECÂNICA,Bacharelado,ARCOS,Turno Noite,Superior,37,22,Redação Online,0,22,2024-09-13 12:00:41.523
...,...,...,...,...,...,...,...,...,...,...,...
1831,ENGENHARIA FLORESTAL,Bacharelado,SÃO JOÃO EVANGELISTA,Turno Manhã e Tarde,Superior,5,1,Notas do Enem,0,1,2024-09-06 20:00:17.127
1832,ADMINISTRAÇÃO,Bacharelado,SÃO JOÃO EVANGELISTA,Turno Noturno,Superior,41,17,Notas do Enem,0,17,2024-09-19 18:00:50.603
1833,TÉCNICO EM INFORMÁTICA,Integrado,SÃO JOÃO EVANGELISTA,Turnos Manhã e Tarde,Médio,87,33,Processo Seletivo,0,33,2024-09-05 22:00:42.270
1834,TÉCNICO EM INFORMÁTICA,Integrado,SÃO JOÃO EVANGELISTA,Turnos Manhã e Tarde,Médio,97,34,Processo Seletivo,0,34,2024-09-07 12:00:37.269


In [21]:
df_amostra[df_amostra["Campus"] == "PONTE NOVA"]

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,FormaIngresso,Isenções deferidas,Inscrições homologadas,Timestamp
1326,ADMINISTRAÇÃO,Integrado,PONTE NOVA,Turnos Manhã e Tarde,médio,73,29,Processo Seletivo,0,29,2024-09-03 15:49:27.870
1327,PROCESSOS GERENCIAIS,Tecnológico,PONTE NOVA,Turno Noturno,Superior,20,6,Redação Online,0,6,2024-09-13 07:15:52.641
1328,ADMINISTRAÇÃO,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,91,29,Processo Seletivo,0,29,2024-09-04 23:40:05.987
1329,PROCESSOS GERENCIAIS,Tecnológico,PONTE NOVA,Turno Noturno,Superior,10,2,Redação Online,0,2,2024-09-03 23:05:14.964
1330,PROCESSOS GERENCIAIS,Tecnológico,PONTE NOVA,Turno Noturno,Superior,27,8,Redação Online,0,8,2024-09-22 23:00:41.604
...,...,...,...,...,...,...,...,...,...,...,...
1423,ADMINISTRAÇÃO,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,123,66,Processo Seletivo,0,66,2024-09-11 18:00:40.656
1424,INFORMÁTICA,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,102,34,Processo Seletivo,0,34,2024-09-03 23:05:14.964
1425,PROCESSOS GERENCIAIS,Tecnológico,PONTE NOVA,Turno Noturno,Superior,20,6,Redação Online,0,6,2024-09-12 06:59:53.332
1426,INFORMÁTICA,Integrado,PONTE NOVA,Turnos Manhã e Tarde,Médio,106,39,Processo Seletivo,0,39,2024-09-04 18:12:57.123
